# NoETL Playbook Execution Report
__Agent 007 mission report:__ _Operation completed. Martini status: shaken, not stirred._
- This notebook provides a report of the execution of a NoETL playbook using the `agent007.py` script.
- It analyzes the DuckDB database generated during the playbook run, with execution flow, step results, loop iterations, task executions, and context data captured throughout the process.
1. Run playbook with `agent007.py`.
2. Set `db_path` below to that file path.
3. Run all paragraphs to inspect tables and data.


In [1]:
%load_ext sql


The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

## List all tables


In [2]:
import duckdb
import polars as pl
import os
import matplotlib.pyplot as plt
import pandas as pd
import json
import networkx as nx
from datetime import datetime

# Path to DuckDB database
db_path = '../data/noetldb/agent007.duckdb'  # Default path used by agent007.py
# Example: db_path = '/Users/kadyapam/projects/noetl/noetl/noetl/runtime/agent007.duckdb'

if not os.path.exists(db_path) and db_path != ':memory:':
    raise FileNotFoundError(f"DuckDB database not found: {db_path}")

con = duckdb.connect(db_path)

def parse_json_column(df, column_name):
    if column_name in df.columns:
        def safe_json_parse(x):
            if not x:
                return None
            try:
                return json.loads(x)
            except json.JSONDecodeError as e:
                print(f"Warning: Could not parse JSON: {e}")
                return x

        return df.with_columns([
            pl.col(column_name).map_elements(safe_json_parse, return_dtype=pl.Object)
        ])
    return df

# execution ID
latest_execution = con.execute("""
    SELECT execution_id, MAX(timestamp) as latest_time
    FROM event_log
    GROUP BY execution_id
    ORDER BY latest_time DESC
    LIMIT 1
""").fetchone()

if latest_execution:
    execution_id = latest_execution[0]
    print(f"Analyzing execution ID: {execution_id}")
else:
    print("No executions found in the database.")
    execution_id = None


Analyzing execution ID: a497e03e-1e8a-4708-97da-cd981e67fde8


In [3]:
tables = con.execute("SHOW TABLES").fetchall()
for t in tables:
    print(t[0])


context
event_log
loop_state
step_result
task_result
transition
workbook
workflow


## Execution Overview

This section provides a high-level overview of the playbook execution.


In [4]:
if execution_id:
    # execution start end times
    execution_times = con.execute("""
        SELECT 
            MIN(timestamp) as start_time,
            MAX(timestamp) as end_time,
            (EXTRACT(EPOCH FROM MAX(timestamp)) - EXTRACT(EPOCH FROM MIN(timestamp))) as duration_seconds
        FROM event_log
        WHERE execution_id = ?
    """, [execution_id]).fetchone()

    # execution statistics
    execution_stats = con.execute("""
        SELECT
            (SELECT COUNT(*) FROM step_result WHERE execution_id = ?) as step_count,
            (SELECT COUNT(*) FROM task_result WHERE execution_id = ?) as task_count,
            (SELECT COUNT(*) FROM loop_state WHERE execution_id = ?) as loop_count,
            (SELECT COUNT(*) FROM event_log WHERE execution_id = ? AND event_type LIKE '%error%') as error_count
    """, [execution_id, execution_id, execution_id, execution_id]).fetchone()

    # playbook name
    playbook_info = con.execute("""
        SELECT node_name, metadata
        FROM event_log
        WHERE execution_id = ? AND event_type = 'execution_start'
        LIMIT 1
    """, [execution_id]).fetchone()

    # execution overview
    print(f"Playbook: {playbook_info[0] if playbook_info else 'Unknown'}")
    print(f"Start Time: {execution_times[0]}")
    print(f"End Time: {execution_times[1]}")
    print(f"Duration: {execution_times[2]:.2f} seconds")
    print(f"Steps Executed: {execution_stats[0]}")
    print(f"Tasks Executed: {execution_stats[1]}")
    print(f"Loops Executed: {execution_stats[2]}")
    print(f"Errors Encountered: {execution_stats[3]}")

    # playbook metadata
    if playbook_info and playbook_info[1]:
        try:
            metadata = json.loads(playbook_info[1])
            print("\nPlaybook Metadata:")
            for key, value in metadata.items():
                print(f"  {key}: {value}")
        except:
            pass
else:
    print("No execution ID available.")


Playbook: weather_iterator_example
Start Time: 2025-06-15 21:06:41.543364
End Time: 2025-06-15 21:06:41.672166
Duration: 0.13 seconds
Steps Executed: 6
Tasks Executed: 11
Loops Executed: 2
Errors Encountered: 0

Playbook Metadata:
  playbook_path: catalog/playbooks/weather_example.yaml


## Step Execution Analysis

Analyze the execution of steps in the playbook.


In [5]:
if execution_id:
    # step execution details
    step_results = pl.from_arrow(con.execute("""
        SELECT 
            step_id, 
            step_name, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM step_result
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    step_results = parse_json_column(step_results, 'data')

    # step execution summary
    step_summary = pl.from_arrow(con.execute("""
        SELECT 
            step_name, 
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM step_result
        WHERE execution_id = ?
        GROUP BY step_name
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Step Execution Summary:")
    display(step_summary)

    # step results
    print("\nStep Results:")
    display(step_results)
else:
    print("No execution ID available.")


Step Execution Summary:


step_name,execution_count,success_count,error_count
str,i64,i64,i64
"""city_loop""",1,1,0
"""aggregate_alerts""",1,1,0
"""end_district_loop""",1,1,0
"""start""",1,1,0
"""district_loop""",1,1,0
"""end_city_loop""",1,1,0



Step Results:


step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,datetime[μs],str,object,str
"""09485942-784e-4a03-987b-26017f…","""start""",null,2025-06-15 21:06:41.545091,"""success""",{},null
"""3f85e793-7aa8-4823-b0c2-7c7533…","""city_loop""",null,2025-06-15 21:06:41.582750,"""success""","[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",null
"""9e5291c6-d005-44f5-97e6-9ee147…","""district_loop""",null,2025-06-15 21:06:41.642809,"""success""","[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",null
"""9a7cf927-81b5-4e07-85f1-100f2c…","""end_district_loop""","""17687414-a5a4-441e-96b9-26b891…",2025-06-15 21:06:41.662884,"""success""",{'district_loop_output': None},null
"""29377228-789a-451c-a19a-4dc332…","""end_city_loop""","""859138b5-0334-43fa-b485-3b0469…",2025-06-15 21:06:41.667908,"""success""",{'alerts': None},null
"""784d7dc1-e8a9-48fd-99dc-7a9d08…","""aggregate_alerts""",null,2025-06-15 21:06:41.670570,"""success""",{},null


## Loop Execution Analysis

Analyze the execution of loops in the playbook.


In [6]:
if execution_id:
    # loop execution details
    loop_state = pl.from_arrow(con.execute("""
        SELECT 
            loop_id, 
            loop_name, 
            parent_id, 
            iterator, 
            items, 
            current_index, 
            current_item, 
            results, 
            timestamp, 
            status
        FROM loop_state
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    loop_state = parse_json_column(loop_state, 'items')
    loop_state = parse_json_column(loop_state, 'current_item')
    loop_state = parse_json_column(loop_state, 'results')

    # loop execution summary
    if len(loop_state) > 0:
        print("Loop Execution Summary:")
        display(loop_state)

        # loop iteration events
        loop_events = pl.from_arrow(con.execute("""
            SELECT 
                event_id, 
                node_name, 
                event_type, 
                timestamp, 
                status, 
                duration, 
                output_result as data, 
                metadata
            FROM event_log
            WHERE execution_id = ? AND event_type LIKE '%loop%'
            ORDER BY timestamp
        """, [execution_id]).arrow())

        loop_events = parse_json_column(loop_events, 'data')
        loop_events = parse_json_column(loop_events, 'metadata')

        print("\nLoop Events:")
        display(loop_events)
    else:
        print("No loop executions found.")
else:
    print("No execution ID available.")


Loop Execution Summary:


loop_id,loop_name,parent_id,iterator,items,current_index,current_item,results,timestamp,status
str,str,str,str,object,i32,object,object,datetime[μs],str
"""17687414-a5a4-441e-96b9-26b891…","""district_loop""","""9e5291c6-d005-44f5-97e6-9ee147…","""district""","[{'name': 'Downtown', 'population': 50000}, {'name': 'North', 'population': 25000}, {'name': 'East', 'population': 30000}, {'name': 'Mordor', 'population': 666}]",4,null,"[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",2025-06-15 21:06:41.662518,"""completed"""
"""859138b5-0334-43fa-b485-3b0469…","""city_loop""","""3f85e793-7aa8-4823-b0c2-7c7533…","""city""","[{'name': 'London', 'lat': 51.51, 'lon': -0.13}, {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}, {'name': 'Berlin', 'lat': 52.52, 'lon': 13.41}]",3,null,"[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",2025-06-15 21:06:41.667661,"""completed"""



Loop Events:


event_id,node_name,event_type,timestamp,status,duration,data,metadata
str,str,str,datetime[μs],str,f64,object,object
"""a5988176-88bf-4102-97d6-53d387…","""city_loop""","""loop_start""",2025-06-15 21:06:41.552318,"""in_progress""",0.0,null,"{'item_count': 3, 'iterator': 'city'}"
"""bf13e022-31e8-4afd-9d5e-34ec4f…","""city_loop[0]""","""loop_iteration""",2025-06-15 21:06:41.555394,"""in_progress""",0.0,null,"{'index': 0, 'item': {'name': 'London', 'lat': 51.51, 'lon': -0.13}}"
"""b243accb-5efa-45e6-82ad-8ce16e…","""city_loop[0]""","""loop_iteration_complete""",2025-06-15 21:06:41.564083,"""success""",0.0,"{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}","{'index': 0, 'item': {'name': 'London', 'lat': 51.51, 'lon': -0.13}}"
"""f1c4f1dd-6739-4d20-96a7-9cd582…","""city_loop[1]""","""loop_iteration""",2025-06-15 21:06:41.564947,"""in_progress""",0.0,null,"{'index': 1, 'item': {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}}"
"""2470dfcd-7c4d-4f85-85c5-7b0375…","""city_loop[1]""","""loop_iteration_complete""",2025-06-15 21:06:41.572231,"""success""",0.0,"{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}","{'index': 1, 'item': {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}}"
…,…,…,…,…,…,…,…
"""dca82db3-7979-411d-a377-58b455…","""district_loop""","""loop_complete""",2025-06-15 21:06:41.643408,"""success""",0.040673,"[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]","{'item_count': 4, 'processed_count': 4}"
"""9f57bb99-e302-4dbc-a35e-4c27d5…","""end_district_loop""","""end_loop_start""",2025-06-15 21:06:41.654824,"""in_progress""",0.0,null,{'loop_name': 'district_loop'}
"""69fc50d9-fcc6-441c-9ce8-1c786b…","""end_district_loop""","""end_loop_complete""",2025-06-15 21:06:41.663337,"""success""",0.008515,{'district_loop_output': None},{'loop_name': 'district_loop'}


## Task Execution Analysis

Analyze the execution of tasks in the playbook.


In [7]:
if execution_id:
    # task execution details
    task_results = pl.from_arrow(con.execute("""
        SELECT 
            task_id, 
            task_name, 
            task_type, 
            parent_id, 
            timestamp, 
            status, 
            data, 
            error
        FROM task_result
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    task_results = parse_json_column(task_results, 'data')

    # task execution summary
    task_summary = pl.from_arrow(con.execute("""
        SELECT 
            task_name, 
            task_type,
            COUNT(*) as execution_count,
            COUNT(CASE WHEN status = 'success' THEN 1 END) as success_count,
            COUNT(CASE WHEN status = 'error' THEN 1 END) as error_count
        FROM task_result
        WHERE execution_id = ?
        GROUP BY task_name, task_type
        ORDER BY execution_count DESC
    """, [execution_id]).arrow())

    print("Task Execution Summary:")
    display(task_summary)

    # task results
    print("\nTask Results:")
    display(task_results)
else:
    print("No execution ID available.")


Task Execution Summary:


task_name,task_type,execution_count,success_count,error_count
str,str,i64,i64,i64
"""process_district""","""python""",5,5,0
"""get_forecast""","""http""",4,4,0
"""alert_task""","""http""",1,1,0
"""get_city_districts""","""http""",1,1,0



Task Results:


task_id,task_name,task_type,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,object,str
"""97b2b1f5-5d36-4228-8b98-ead35d…","""get_forecast""","""http""","""154ecf5f-f0e2-4701-8070-c7dfbe…",2025-06-15 21:06:41.559646,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""db3feb39-0b0c-4961-b35d-9ce76a…","""get_forecast""","""http""","""9ca98b4b-81e4-4eab-9e35-b5eb95…",2025-06-15 21:06:41.568933,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""5144df5a-931f-4520-a85e-f07d23…","""get_forecast""","""http""","""d2223d08-0628-4b07-94c5-c73f83…",2025-06-15 21:06:41.577582,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""ea7753ff-5e81-4e3b-bf45-fc8dfe…","""get_forecast""","""http""","""9f1ccc49-f64d-4524-9bbe-b11abd…",2025-06-15 21:06:41.587114,"""success""","{'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}",null
"""8138d169-32b2-408f-b841-ad94b3…","""alert_task""","""http""","""eb8e1004-7159-4e9e-878b-082d84…",2025-06-15 21:06:41.593841,"""success""","{'data': 'mocked_response', 'status': 'success'}",null
…,…,…,…,…,…,…,…
"""d6a7628b-ad54-45d7-a9ba-239117…","""process_district""","""python""","""5b9705fb-2219-4b3a-8fec-90bb5c…",2025-06-15 21:06:41.609209,"""success""","{'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null
"""5915873b-0ea7-44b1-9435-2a8664…","""process_district""","""python""","""17244123-7cef-40d4-a266-ffaf2e…",2025-06-15 21:06:41.618389,"""success""","{'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null
"""7e05a9e5-a592-45f3-9881-e9b322…","""process_district""","""python""","""e6621252-323f-4e2b-b8a2-15ec15…",2025-06-15 21:06:41.627082,"""success""","{'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",null


## Context Analysis

Analyze the context data during playbook execution.


In [8]:
if execution_id:
    # context data
    context_data = pl.from_arrow(con.execute("""
        SELECT 
            key, 
            value, 
            timestamp
        FROM context
        WHERE execution_id = ?
        ORDER BY timestamp
    """, [execution_id]).arrow())

    context_data = parse_json_column(context_data, 'value')

    if len(context_data) > 0:
        print("Context Data:")
        display(context_data)

        # context state
        final_context = {}
        for _, row in context_data.to_pandas().iterrows():
            final_context[row['key']] = row.get('value')

        print("\nFinal Context State:")
        for key, value in final_context.items():
            print(f"\n{key}:")
            if isinstance(value, (dict, list)):
                print(json.dumps(value, indent=2))
            else:
                print(value)
    else:
        print("No context data found.")
else:
    print("No execution ID found.")


Context Data:


key,value,timestamp
str,object,datetime[μs]
"""workload""","{'jobId': '{{ job.uuid }}', 'state': 'ready', 'cities': [{'name': 'London', 'lat': 51.51, 'lon': -0.13}, {'name': 'Paris', 'lat': 48.85, 'lon': 2.35}, {'name': 'Berlin', 'lat': 52.52, 'lon': 13.41}], 'base_url': 'https://api.open-meteo.com/v1', 'temperature_threshold': 25}",2025-06-15 21:06:41.542348
"""execution_start""",2025-06-15T21:06:41.542968,2025-06-15 21:06:41.542976
"""base_url""",https://api.open-meteo.com/v1,2025-06-15 21:06:41.574027
"""temperature_threshold""",25,2025-06-15 21:06:41.574299
"""city_loop_results""","[{'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}, {'fetch_and_evaluate': {'data': {'hourly': {'temperature_2m': [20, 22, 25, 28, 30, 26, 24], 'precipitation_probability': [0, 10, 20, 30, 20, 10, 0], 'windspeed_10m': [10, 12, 15, 18, 22, 19, 14]}}, 'alert': True, 'max_temp': 30}}]",2025-06-15 21:06:41.582448
…,…,…
"""district_loop_results""","[{'process_district': {'city': 'Berlin', 'district': 'Downtown', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'North', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'East', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}, {'process_district': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}}]",2025-06-15 21:06:41.642479
"""process_district""","{'result': {'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}, 'status': 'success'}",2025-06-15 21:06:41.650599
"""result""","{'city': 'Berlin', 'district': 'Mordor', 'processed': True, 'timestamp': '2024-01-01T12:00:00Z'}",2025-06-15 21:06:41.650972



Final Context State:

workload:
{
  "jobId": "{{ job.uuid }}",
  "state": "ready",
  "cities": [
    {
      "name": "London",
      "lat": 51.51,
      "lon": -0.13
    },
    {
      "name": "Paris",
      "lat": 48.85,
      "lon": 2.35
    },
    {
      "name": "Berlin",
      "lat": 52.52,
      "lon": 13.41
    }
  ],
  "base_url": "https://api.open-meteo.com/v1",
  "temperature_threshold": 25
}

execution_start:
2025-06-15T21:06:41.542968

base_url:
https://api.open-meteo.com/v1

temperature_threshold:
25

city_loop_results:
[
  {
    "fetch_and_evaluate": {
      "data": {
        "hourly": {
          "temperature_2m": [
            20,
            22,
            25,
            28,
            30,
            26,
            24
          ],
          "precipitation_probability": [
            0,
            10,
            20,
            30,
            20,
            10,
            0
          ],
          "windspeed_10m": [
            10,
            12,
            

## Preview `event_log` table


In [9]:
table_name = 'event_log'
df = pl.from_arrow(con.execute(f"SELECT * FROM {table_name} LIMIT 20").arrow())
df


execution_id,event_id,parent_event_id,timestamp,event_type,node_id,node_name,node_type,status,duration,input_context,output_result,metadata
str,str,str,datetime[μs],str,str,str,str,str,f64,str,str,str
"""a497e03e-1e8a-4708-97da-cd981e…","""11d58c32-68b0-45e6-818a-295d7a…",null,2025-06-15 21:06:41.543364,"""execution_start""","""a497e03e-1e8a-4708-97da-cd981e…","""weather_iterator_example""","""playbook""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""playbook_path"": ""catalog/pla…"
"""a497e03e-1e8a-4708-97da-cd981e…","""0992c224-020a-40d0-ad9d-836927…",null,2025-06-15 21:06:41.544493,"""step_start""","""09485942-784e-4a03-987b-26017f…","""start""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""bf7026af-a965-496e-99cf-eda7f0…","""0992c224-020a-40d0-ad9d-836927…",2025-06-15 21:06:41.545576,"""step_complete""","""09485942-784e-4a03-987b-26017f…","""start""","""step""","""success""",0.001186,"""{""jobId"": ""{{ job.uuid }}"", ""s…","""{}""","""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""6193ea67-8155-4e2b-b6be-4de1b3…","""11d58c32-68b0-45e6-818a-295d7a…",2025-06-15 21:06:41.549975,"""step_transition""","""a497e03e-1e8a-4708-97da-cd981e…","""transition_to_city_loop""","""transition""","""success""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""from_step"": ""start"", ""to_ste…"
"""a497e03e-1e8a-4708-97da-cd981e…","""0991095f-57c4-45a8-b17f-c0c999…",null,2025-06-15 21:06:41.551111,"""step_start""","""3f85e793-7aa8-4823-b0c2-7c7533…","""city_loop""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""a497e03e-1e8a-4708-97da-cd981e…","""f6bfa8c6-a980-48d2-b7af-13205d…",null,2025-06-15 21:06:41.566602,"""step_start""","""575c80c1-a509-4420-b63d-b432e3…","""fetch_and_evaluate""","""step""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""step_type"": ""standard""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""9ca98b4b-81e4-4eab-9e35-b5eb95…","""f6bfa8c6-a980-48d2-b7af-13205d…",2025-06-15 21:06:41.567217,"""task_execute""","""b1764051-4ec0-45f0-bb10-fdc9f6…","""get_forecast""","""task.http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""task_type"": ""http""}"""
"""a497e03e-1e8a-4708-97da-cd981e…","""9b6dba54-6c82-4610-879f-b9d031…","""9ca98b4b-81e4-4eab-9e35-b5eb95…",2025-06-15 21:06:41.568354,"""task_start""","""db3feb39-0b0c-4961-b35d-9ce76a…","""get_forecast""","""http""","""in_progress""",0.0,"""{""jobId"": ""{{ job.uuid }}"", ""s…",null,"""{""method"": ""GET"", ""endpoint"": …"


## Show all steps resluts


In [10]:
pl.from_arrow(con.execute("SELECT * FROM step_result ORDER BY timestamp DESC LIMIT 10").arrow())


execution_id,step_id,step_name,parent_id,timestamp,status,data,error
str,str,str,str,datetime[μs],str,str,str
"""a497e03e-1e8a-4708-97da-cd981e…","""784d7dc1-e8a9-48fd-99dc-7a9d08…","""aggregate_alerts""",null,2025-06-15 21:06:41.670570,"""success""","""{}""",null
"""a497e03e-1e8a-4708-97da-cd981e…","""29377228-789a-451c-a19a-4dc332…","""end_city_loop""","""859138b5-0334-43fa-b485-3b0469…",2025-06-15 21:06:41.667908,"""success""","""{""alerts"": null}""",null
"""a497e03e-1e8a-4708-97da-cd981e…","""9a7cf927-81b5-4e07-85f1-100f2c…","""end_district_loop""","""17687414-a5a4-441e-96b9-26b891…",2025-06-15 21:06:41.662884,"""success""","""{""district_loop_output"": null}""",null
"""a497e03e-1e8a-4708-97da-cd981e…","""9e5291c6-d005-44f5-97e6-9ee147…","""district_loop""",null,2025-06-15 21:06:41.642809,"""success""","""[{""process_district"": {""city"":…",null
"""a497e03e-1e8a-4708-97da-cd981e…","""3f85e793-7aa8-4823-b0c2-7c7533…","""city_loop""",null,2025-06-15 21:06:41.582750,"""success""","""[{""fetch_and_evaluate"": {""data…",null
"""a497e03e-1e8a-4708-97da-cd981e…","""09485942-784e-4a03-987b-26017f…","""start""",null,2025-06-15 21:06:41.545091,"""success""","""{}""",null


## Show all task results

In [11]:
con.sql("SELECT * FROM task_result ORDER BY timestamp DESC LIMIT 10").show()

┌──────────────────────────────────────┬──────────────────────────────────────┬────────────────────┬───────────┬──────────────────────────────────────┬────────────────────────────┬─────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────┐
│             execution_id             │               task_id                │     task_name      │ task_type │              parent_id               │         timestamp          │ status  │                                                                                                     data                                                                                                      │  error  │
│               varchar                │               varchar                │      varchar       │  varchar  │               varchar                │         timestamp          │ v

In [12]:
%sql con --alias duckdb
%sql SELECT * FROM task_result ORDER BY timestamp DESC LIMIT 10

Running query in 'duckdb'

execution_id,task_id,task_name,task_type,parent_id,timestamp,status,data,error
a497e03e-1e8a-4708-97da-cd981e67fde8,86f2006f-a18c-44d1-a617-4a35b474039b,process_district,python,5a14bb1b-a100-4946-be87-d67dc8f67c84,2025-06-15 21:06:41.648569,success,"{""city"": ""Berlin"", ""district"": ""Mordor"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,3cca606a-b8d8-4d55-9cfe-e412706c1683,process_district,python,130be37f-34cf-4934-855a-57eb0ec5f21e,2025-06-15 21:06:41.637150,success,"{""city"": ""Berlin"", ""district"": ""Mordor"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,7e05a9e5-a592-45f3-9881-e9b322cad739,process_district,python,e6621252-323f-4e2b-b8a2-15ec1566c357,2025-06-15 21:06:41.627082,success,"{""city"": ""Berlin"", ""district"": ""East"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,5915873b-0ea7-44b1-9435-2a8664e3b016,process_district,python,17244123-7cef-40d4-a266-ffaf2edfc406,2025-06-15 21:06:41.618389,success,"{""city"": ""Berlin"", ""district"": ""North"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,d6a7628b-ad54-45d7-a9ba-239117a8081f,process_district,python,5b9705fb-2219-4b3a-8fec-90bb5cc53605,2025-06-15 21:06:41.609209,success,"{""city"": ""Berlin"", ""district"": ""Downtown"", ""processed"": true, ""timestamp"": ""2024-01-01T12:00:00Z""}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,8be2ea4e-fb8b-44ee-8260-be0f36e30b43,get_city_districts,http,97218c4b-b2ef-44df-a4e8-51398ea57053,2025-06-15 21:06:41.598793,success,"{""data"": [{""name"": ""Downtown"", ""population"": 50000}, {""name"": ""North"", ""population"": 25000}, {""name"": ""East"", ""population"": 30000}, {""name"": ""Mordor"", ""population"": 666}]}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,8138d169-32b2-408f-b841-ad94b3722292,alert_task,http,eb8e1004-7159-4e9e-878b-082d847ab9ec,2025-06-15 21:06:41.593841,success,"{""data"": ""mocked_response"", ""status"": ""success""}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,ea7753ff-5e81-4e3b-bf45-fc8dfe0650ea,get_forecast,http,9f1ccc49-f64d-4524-9bbe-b11abd43400c,2025-06-15 21:06:41.587114,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,5144df5a-931f-4520-a85e-f07d23c774b6,get_forecast,http,d2223d08-0628-4b07-94c5-c73f83a081e3,2025-06-15 21:06:41.577582,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
a497e03e-1e8a-4708-97da-cd981e67fde8,db3feb39-0b0c-4961-b35d-9ce76a86d6f8,get_forecast,http,9ca98b4b-81e4-4eab-9e35-b5eb95ade3b5,2025-06-15 21:06:41.568933,success,"{""data"": {""hourly"": {""temperature_2m"": [20, 22, 25, 28, 30, 26, 24], ""precipitation_probability"": [0, 10, 20, 30, 20, 10, 0], ""windspeed_10m"": [10, 12, 15, 18, 22, 19, 14]}}, ""alert"": true, ""max_temp"": 30}",None
